### 排序模型
通过召回的操作，我们已经进行了问题规模的缩减，对于每个用户，选择出了N篇文章作为了候选集，并基于召回的候选集构建了与用户历史相关的特征，以及用户本身的属性特征，文章本身的属性特征，以及用户与文章之间的特征，下面就是使用机器学习模型来对构造好的特征进行学习，然后对测试集进行预测，得到测试集中的每个候选集用户点击的概率，返回点击概率最大的topk个文章，作为最终的结果。</br>
排序阶段选择了三个比较有代表性的排序模型，它们分别是：</br>

1. LGB的排序模型
2. LGB的分类模型
3. 深度学习的分类模型DIN</br>

得到了最终的排序模型输出的结果之后，还选择了两种比较经典的模型集成的方法：</br>

1. 输出结果加权融合
2. Staking（将模型的输出结果再使用一个简单模型进行预测）

In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = './data/'
save_path = './temp_results/'
offline = False

In [ ]:
trn_user_item_feats_df = pd.read_csv(save_path + 'trn_user_item_feats_df.csv')
trn_user_item_feats_df['click_article_id'] = trn_user_item_feats_df['click_article_id'].astype(int)

if offline:
    val_